##Test TikTokScalping

https://www.tiktok.com/@eduscalping/video/7358462952246103302

In [132]:
import pandas as pd
import numpy as np
from datetime import timedelta

import os
import pickle
from config import BASE_DIR
print(os.getcwd())
os.chdir(BASE_DIR)
import scripts.functions as fn 
from scripts.indicators import find_pivots

from scipy.signal import savgol_filter

C:\Users\lbisa\Mi unidad (leonardo.bisaro@gmail.com)\Cripto\python\trd_app


#Desicion de operacion en Largo
- RSI inferior a 35, sobre de 45, aumentando constante
- Estocastico Inferior 20 y sobre 35, aumentando constante
- MACD cruzo hacia arriba

#Desicion de operacion en Corto
- RSI Supero 65, por debajo de 55, disminuyendo constante
- Estocastico Supero 80 y por debajo de 75, disminuyendo constante
- MACD cruzo hacia abajo

In [133]:
klines_file = './backtest/klines/0m05/Lateral_TRXUSDT_0m05_2023-06-20_2023-10-09.DataFrame'
klines_file  = './backtest/klines/1h01/Completo_BTCUSDT_1h01_2021-01-01_2023-07-31.DataFrame'
klines_file  = './backtest/klines/1h04/Completo_BTCUSDT_1h04_2021-01-01_2023-07-31.DataFrame'

klines_file  = './backtest/klines/2d01/Full_BTCUSDT_2d01_2019-01-01_2024-06-19.DataFrame'

with open(klines_file, 'rb') as file:
    df = pickle.load(file)
    #data.set_index('datetime',inplace=True)

#df = fn.ohlc_mirror_v(df)

#ini = pd.to_datetime('2023-08-03')
#fin = pd.to_datetime('2023-08-08')
#df = df[(df['datetime']>=ini)&(df['datetime']<=fin)]
df.drop(columns=['volume'],inplace=True)


In [134]:
df['price'] = (df['open']+df['close']+df['high']+df['low'])
df['price'] = df['close']
df['price_smooth'] = savgol_filter(df['price'], window_length=14, polyorder=3)
df['price_smooth'] = savgol_filter(df['price'], window_length=14, polyorder=3)

df['pct_change'] = np.diff(df['price_smooth']) / df['price_smooth'][:-1]


df['last_max'] = None
df['last_min'] = None
df['buy'] = None
df['sell'] = None

df = find_pivots(df,dev_threshold=3)
pre_row = None  
for index, row in df.iterrows():
    if pre_row is not None:
        df.at[index, 'last_max'] = pre_row['last_max']
        df.at[index, 'last_min'] = pre_row['last_min']

        if row['max_pivots']>0:
            df.at[index, 'last_max'] = row['max_pivots']

        if row['min_pivots']>0:
            if (pre_row['last_min'] is not None and pre_row['last_max'] is not None and  
                row['min_pivots'] > pre_row['last_min'] and pre_row['last_max'] > row['min_pivots']):
                    df.at[index, 'buy'] = row['close']
            if  pre_row['last_min'] is not None and row['min_pivots'] < pre_row['last_min']:
                df.at[index, 'sell'] = row['close']
            df.at[index, 'last_min'] = row['min_pivots']
        
    pre_row = df.iloc[index]  

#df[df['datetime']>='2020-09-15']

In [135]:
indicators = [
     {'col': 'price','name': 'Price','color': 'gray','row': 0,},
     {'col': 'last_max','name': 'last_max','color': 'green','row': 0,},
     {'col': 'last_min','name': 'last_min','color': 'red','row': 0,},
        ]

events = [
        {'df':df,
         'col':'buy',
         'name': 'BUY',
         'color': 'green',
         'symbol': 'circle-dot' #https://plotly.com/python/reference/scatter/#scatter-marker-symbol
        },
        {'df':df,
         'col':'sell',
         'name': 'SELL',
         'color': 'red',
         'symbol': 'circle-dot' #https://plotly.com/python/reference/scatter/#scatter-marker-symbol
        },
     ]




fig = fn.ohlc_chart(df,indicators=indicators,events=events) #
fig.show()